In [1]:
import pandas as pd

# from features import
from utils import helper_funtions

In [2]:
pd.options.display.max_columns = None  # Remove "dots" from display when printing dataframes

In [3]:
PATH = helper_funtions.get_path()

# Read data

In [4]:
df = pd.read_csv(PATH + 'data/data_cleaning.csv')
df.sample(n=5).head()

,UUID_client,Age,Location,Income,TAX,Previous_sales,Type_products,Contact_channel,Contact_hour,Num_contacts,Satisfaction_score,Sales
532,cc074a02-0660-5673-882f-e651c74101fc,43,CA,79145.0,11871.75,1,C,Email,14:00:00,2,5,1
182,ca01b83d-37a3-5903-b8b5-e619bfb5bc55,46,CA,88348.0,13252.20,0,B,Online Chat,15:00:00,2,1,0
624,0d0d5588-809f-5b79-9401-0a4e431c0d2d,42,NY,66060.0,9909.00,1,A,Email,04:15:00,7,2,0
987,59534e67-94a0-5c97-922f-ea54936a6b18,20,TX,39885.0,5982.75,0,C,Email,10:15:00,4,5,0
124,41bb8d7a-e096-590e-a0e0-18272be62d34,59,FL,95838.0,14375.70,0,C,Phone,05:30:00,5,1,0


# Preprocesing

In [6]:
# Determine peak times by rounding the time in groups of one hour 
df['Contact_hour'] = pd.to_datetime(df['Contact_hour'], format='%H:%M:%S')
df['Contact_hour_round'] = df['Contact_hour'].apply(lambda x: x.round('60 min'))

df_peak_time = df['Contact_hour_round'].value_counts().head()
df_peak_time = df_peak_time.rename_axis('Hour')
df_peak_time = df_peak_time.reset_index(name='Count')
peak_hours = []
for i in range(len(df_peak_time)):
    peak_hours.append(df_peak_time['Hour'].iloc[i])

for i in range(len(df)):
    if df.loc[i, 'Contact_hour_round'] in peak_hours:
        df.at[i, 'Is_peak_hour'] = 1
    else:
        df.at[i, 'Is_peak_hour'] = 0

del df_peak_time

# Save

In [ ]:
# df.to_csv(PATH + 'data/data_preprocesing.csv', index=False, encoding='utf-8')

In [7]:
df.sample(n=5).head()

,UUID_client,Age,Location,Income,TAX,Previous_sales,Type_products,Contact_channel,Contact_hour,Num_contacts,Satisfaction_score,Sales,Contact_hour_round,Is_peak_hour
552,6978bfe8-2184-5afc-a914-3a90cc8c6adf,40,TX,73541.0,11031.15,1,B,Online Chat,1900-01-01 09:45:00,1,4,1,1900-01-01 10:00:00,1.0
903,f2d1b062-f2fc-53cc-b8db-5c795b440395,25,CA,70718.0,10607.70,0,B,Phone,1900-01-01 09:45:00,3,2,0,1900-01-01 10:00:00,1.0
235,66caf453-a948-5c48-a4fe-e1b3736783f9,58,IL,61910.0,9286.50,2,B,Phone,1900-01-01 17:15:00,3,2,0,1900-01-01 17:00:00,0.0
471,39a2eb00-d06f-5906-adb9-3e29f7e4e263,41,TX,37099.0,5564.85,0,B,Phone,1900-01-01 04:30:00,2,2,0,1900-01-01 04:00:00,0.0
562,71b89201-974e-58ae-8aa0-95dc4b7cef7c,34,NY,50567.0,7585.05,1,A,Email,1900-01-01 05:45:00,4,3,0,1900-01-01 06:00:00,1.0
